<a href="https://colab.research.google.com/github/BrunoBustos96/DeteccionSomnolenciaCNN/blob/main/1_Detector_De_Somnoliencia_PrimeraParte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Detector de somnoliencia con Redes Neuronales Convolucionales CNN**

Primera parte de tres partes


# **Recolección y preparación de datos** 
## 1. Dataset
En esta primera parte para la aplicación de Redes Neuronales Convolucionales para la detecctión de somnoliencia, nos enfocamos en la obtención del dataset de acuerdo a nuestros requerimientos. El dataset recolectado proviene de:

* LFW dataset (para caras con ojos abiertos): Labeled Faces in the Wild (LFW) es un conjunto de datos de imágenes que contiene fotografías de caras, recogidas especialmente para estudiar el problema del reconocimiento de caras sin restricciones. Incluye más de 13.000 imágenes de caras recogidas en todo el mundo. Link: http://vis-www.cs.umass.edu/lfw/
* CEW dataset (para caras con ojos cerrados) Link: http://parnec.nuaa.edu.cn/_upload/tpl/02/db/731/template731/pages/xtan/ClosedEyeDatabases.html 

## 2. Preparación del dataset

* En esta sección nos enfocaremos en utilizar una función que nos permita el recorte de la sección de los ojos por separado de la cara completa. En otras palabras, tomaremos una imagen de cara completa y extraeremos el ojo izquierdo y derecho de la imagen. Cada una de estas imágenes recortadas serán almacenadas en el Google Drive, para después ser utilizadas para el entrenamiento de los modelos de nuestro proyecto.

Tras finalizar este proceso se revisaron las imágenes recortadas para verificar que se recortaron tanto ojos derecho e izquierdos. Este resultado obtenido se convertirá en un archivo .zip




## **1. Importamos las librerías**

In [ ]:
#Instalación de la librería de face_recognition
!pip install face_recognition

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 100.1 MB 23 kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566186 sha256=1c67d48554abd229f00ecf468ef3459a71e766203a546a3f38bb7cb99efff2b2
  Stored in directory: /root/.cache/pip/wheels/d6/81/3c/884bcd5e1c120ff548d57c2ecc9ebf3281c9a6f7c0e7e7947a
Successfully built face-recognition-models


In [ ]:
# OS nos permite usar funcionalidades dependientes del sistema operativo, lectura, escritura de archivos, manipulación de rutas.
import os
#PIL permite la edición de imágenes desde Python
#Image provee funciones para cargar imágenes desde archivos
#ImageDraw provee funcionalidades de graficos en 2D en objetos de Image, creando nuevas imágenes, retoque y anotación de imágenes 
from PIL import Image, ImageDraw
#Se importa la librería del módulo previamente instalado, el cual permite el reconocimiento de rostros en Python
import face_recognition

## **2. Montamos la unidad para acceder a los archivos de la nube del Drive**

In [ ]:
# Montar Google Drive en tiempo de ejecución utilizando un código de autorización, escribir y leer archivos allí.
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


## **3. Creación de las funciones de recorte de ojos**

Estas funciones toman imágenes de rostros completos y recortan los ojos de esa imagen. A continuación, las almacena en la carpeta de datos "train" para utilizarlas en el entrenamiento y ajuste del modelo.

### 3.1 Función para recortar imagen de ojos abiertos  

In [ ]:
#Función para recortar imagen de ojos abiertos  

def eye_cropper(folders):
    
    # Establecemos contador (count) para guardar archivo de manera iterativa
    count = 0

    # Usamos el bucle FOR para recorrer todos los archivos de imagen 
    for folder in os.listdir(folders):
        for file in os.listdir(folders + '/' + folder):
          
            # Se utiliza la librería face_recognition en las imágenes
            image = face_recognition.load_image_file(folders + '/' + folder + '/' + file)
          
            # Crear variables para determinar las coordenadas de los rasgos faciales 
            face_landmarks_list = face_recognition.face_landmarks(image)

            # Se crea un array para marcar la posición de las coordenadas de los ojos 
            # y añadir las coordenadas de los ojos que no hayan sido encontrados 
            eyes = []
            try:
                eyes.append(face_landmarks_list[0]['left_eye'])
                eyes.append(face_landmarks_list[0]['right_eye'])
            except:
                continue

            # Establecemos las coordenadas X y Y maximas de los ojos
            for eye in eyes:
                x_max = max([coordinate[0] for coordinate in eye])
                x_min = min([coordinate[0] for coordinate in eye])
                y_max = max([coordinate[1] for coordinate in eye])
                y_min = min([coordinate[1] for coordinate in eye])

              # Establecemos el rango de las coordenadas X y Y    
                x_range = x_max - x_min
                y_range = y_max - y_min
              
              # Para asegurar que el ojo entero es capturado, calculamos las coordenadas de un cuadrado que tenga
              # 50% de brecha al eje mayor y hacerlo coincidir el rango menor con el rango mayor

                if x_range > y_range:
                    right = round(.5*x_range) + x_max
                    left = x_min - round(.5*x_range)
                    bottom = round(((right-left) - y_range))/2 + y_max
                    top = y_min - round(((right-left) - y_range))/2
                else:
                    bottom = round(.5*y_range) + y_max
                    top = y_min - round(.5*y_range)
                    right = round(((bottom-top) - x_range))/2 + x_max
                    left = x_min - round(((bottom-top) - x_range))/2
              
                # Guardamos la imagen original como una variable
                im = Image.open(folders + '/' + folder + '/' + file)
                
                # Cortamos la imagen original usando las coordenadas
                im = im.crop((left, top, right, bottom))
              
                # Redimensionamos la imagen para agregar a nuestro modelo
                im = im.resize((80,80))
              
                # Guardar archivo a la carpeta de salida
                # Actualizar la carpeta donde se almacenan las imagenes recortadas de los ojos
                im.save('/content/drive/MyDrive/ProyectoDrowsinessDectection/datasets/data/train/Ojos_Abiertos/eye_crop_open' + str(count) + '.jpg')
              
                # Incrementamos el contador al guardar el archivo de manera iterativa
                count += 1
              
                # Imprimimos el contador cada 100 fotos guardadas para monitorear el proceso
                if count % 100 == 0:
                    print(count)
    
# Se llama a la función para cortar el rostro entero con ojos abiertos
eye_cropper('/content/drive/MyDrive/ProyectoDrowsinessDectection/datasets/data/Datos_Ojos_Abiertos_Caras') 

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100


### 3.2 Función para recortar imágenes de ojos cerrados

In [ ]:
# Creando el directorio data, copiando y sacando la información del archivo zip que contiene los datos de las imágenes de caras con ojos cerrados
!mkdir data
!cp /content/drive/MyDrive/DetectorSomnoliencia/Data/Datos_Ojos_Cerrados_Caras.zip data
!cd data && unzip Datos_Ojos_Cerrados_Caras.zip

Archive:  Datos_Ojos_Cerrados_Caras.zip
  inflating: Datos_Ojos_Cerrados_Caras/closed_eye_1307.jpg_face_1.jpg  
  inflating: Datos_Ojos_Cerrados_Caras/closed_eye_0791.jpg_face_2.jpg  
  inflating: Datos_Ojos_Cerrados_Caras/closed_eye_1853.jpg_face_1.jpg  
  inflating: Datos_Ojos_Cerrados_Caras/closed_eye_2446.jpg_face_1.jpg  
  inflating: Datos_Ojos_Cerrados_Caras/closed_eye_1646.jpg_face_1.jpg  
  inflating: Datos_Ojos_Cerrados_Caras/closed_eye_0231.jpg_face_3.jpg  
  inflating: Datos_Ojos_Cerrados_Caras/closed_eye_0232.jpg_face_2.jpg  
  inflating: Datos_Ojos_Cerrados_Caras/closed_eye_0463.jpg_face_1.jpg  
  inflating: Datos_Ojos_Cerrados_Caras/closed_eye_1381.jpg_face_1.jpg  
  inflating: Datos_Ojos_Cerrados_Caras/closed_eye_1773.jpg_face_1.jpg  
  inflating: Datos_Ojos_Cerrados_Caras/closed_eye_1249.jpg_face_1.jpg  
  inflating: Datos_Ojos_Cerrados_Caras/closed_eye_0727.jpg_face_1.jpg  
  inflating: Datos_Ojos_Cerrados_Caras/closed_eye_1486.jpg_face_4.jpg  
  inflating: Datos_Ojos_

In [ ]:
#Función para recortar imágenes de ojos cerrados  

def eye_cropper(folder):
    
    # Establecemos el contador (count) para guardar archivo de manera iterativa
    count = 0

    # Usamos el bucle FOR para recorrer todos los archivos de imagen 
    for file in os.listdir(folder):
        
        # Se utiliza la librería face_recognition en las imágenes
        image = face_recognition.load_image_file(folder + '/' + file)
        
        # Creamos variable para determinar las coordenadas de los rasgos faciales 
        face_landmarks_list = face_recognition.face_landmarks(image)
        
        # Se crea un array para marcar la posición de las coordenadas de los ojos 
        # y añadir las coordenadas de los ojos al array a menos que no hayan sido reconocidos por face_recognition 
        eyes = []
        try:
            eyes.append(face_landmarks_list[0]['left_eye'])
            eyes.append(face_landmarks_list[0]['right_eye'])
        except:
            continue

        # Establecemos las coordenadas X y Y máximas de los ojos
        for eye in eyes:
            x_max = max([coordinate[0] for coordinate in eye])
            x_min = min([coordinate[0] for coordinate in eye])
            y_max = max([coordinate[1] for coordinate in eye])
            y_min = min([coordinate[1] for coordinate in eye])

        # Establecemos el rango de las coordenadas X y Y    
            x_range = x_max - x_min
            y_range = y_max - y_min
            
            
            # Para asegurar que el ojo entero es capturado, calcular las coordenadas de un cuadrado que tenga
            # 50% de brecha al eje mayor y hacerlo coincidir el rango menor con el rango mayor de la brecha

            if x_range > y_range:
                right = round(.5*x_range) + x_max
                left = x_min - round(.5*x_range)
                bottom = round(((right-left) - y_range))/2 + y_max
                top = y_min - round(((right-left) - y_range))/2
            else:
                bottom = round(.5*y_range) + y_max
                top = y_min - round(.5*y_range)
                right = round(((bottom-top) - x_range))/2 + x_max
                left = x_min - round(((bottom-top) - x_range))/2
            
            # Guardamos la imagen original como una variable
            im = Image.open(folder + '/' + file)
            
            # Recortamos la imagen original usando las coordenadas de la brecha
            im = im.crop((left, top, right, bottom))
            
            # Redimensionamos la imagen para agregar a nuestro modelo
            im = im.resize((80,80))
            
            # Guardamos el archivo a la carpeta de salida
            # Actualizamos la carpeta donde se almacenan las imagenes recortadas de los ojos
            im.save('/content/drive/MyDrive/DetectorSomnoliencia/Datasets/data/train/Ojos_Cerrados/eye_crop_closed' + str(count) + '.jpg')
            
            # Incrementamos contador al guardar el archivo de manera iterativa
            count += 1
            
            # Imprimir contador cada 10 fotos guardadas para monitorear el proceso
            if count % 10 == 0:
                print(count)

# Se llama a la función para cortar el rostro entero y de ojos cerrados
eye_cropper('/content/data/Datos_Ojos_Cerrados_Caras')  

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
